# Notebook to find scraped charity names in the Panama Papers using Spark, specifically targeting officer nodes

In [1]:
# Imports
import re
import nltk
import json
import folium
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#stop words
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

#spark
import findspark
#findspark.init(r'C:\Users\Ruijia\Spark')

#Sabrina
findspark.init('/opt/spark/spark-2.3.2-bin-hadoop2.7/')


from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.functions import min
from pyspark.sql.functions import udf
from pyspark.sql.functions import split
from pyspark.sql.functions import explode

from pyspark.sql.types import StringType
from pyspark.sql.types import TimestampType

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.getOrCreate()


In [2]:
# Addition of english stop words

def init_stopwords():
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

    stop_words.add('&')
    stop_words.add('co')
    stop_words.add('co.')
    stop_words.add('co.,')
    stop_words.add('co.,ltd.')
    stop_words.add('corp')
    stop_words.add('corp.')
    stop_words.add('corp.,')
    stop_words.add('de')
    stop_words.add('foundation')
    stop_words.add('inc')
    stop_words.add('inc.')
    stop_words.add('limited')
    stop_words.add('international')
    stop_words.add('ltd')
    stop_words.add('ltd.')
    stop_words.add('s.a.')
    stop_words.add('world')
    stop_words.add('global')

    stop_words = list(stop_words)
    return stop_words

stop_words = init_stopwords()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def check_for_words(charity, shell, stop_words, tuning):
    
    percentage = 0.6
    
    if charity is None or shell is None:
        return False
    
    charity_words = [x.lower() for x in charity.split()]
    shell_words = [x.lower() for x in shell.split()]
    len_charity = len(charity_words)
    len_shell = len(shell_words)
    
    if len_charity == 0 or len_shell == 0:
        return False
    
    count_random_matches = 0
    stop_word_random_matches = 0
    
    for i in range(len_charity):
        word = charity_words[i]
        if word in shell_words:
            count_random_matches += 1
            
            if word in stop_words:
                stop_word_random_matches += 1
                
    if tuning:
        #if only stopwords match, not valid
        if count_random_matches - stop_word_random_matches < 1:
            return False

        #"Family foundations are tricky -> make sure they are not the only matching parts"
        if ('family' in shell_words 
            and 'foundation' in shell_words 
            and 'family' in charity_words 
            and 'foundation' in charity_words 
            and count_random_matches < 3 
            and len_shell > 2 
            and len_charity > 2):
            return False

    if len_charity == 1 or len_shell == 1:
        return (np.abs(len_charity - len_shell) < 2  and count_random_matches == 1)
        
    return ((count_random_matches/len_charity >= percentage) 
            and (count_random_matches/len_shell >= percentage))
        

In [13]:
def extract_matches_between(leak, charity, sharp):
    
    stop_words = init_stopwords()
    
    charity_location = '../generated/' + charity + '/' + charity + '_charity_info.csv'
    leak_location = '../data/' + leak + '/' + leak + '*.nodes.officer.csv'
    
    leak_data = spark.read.csv(leak_location, header=True)

    charity_data = spark.read.csv(charity_location, header=True)
    
    charity_names = charity_data.select('name', 'Headquarters').withColumnRenamed('name', 'CharityName')
    shell_names = leak_data.select('node_id','name').withColumnRenamed('name', 'ShellName')
    
    shells_vs_charities = shell_names.crossJoin(charity_names)
    
    filtered_names = shells_vs_charities.rdd.filter(lambda r: check_for_words(r[1], r[2], stop_words, sharp) == True)
    
    
    
    matches = pd.DataFrame(filtered_names.collect(), columns=['node_id','ShellName','CharityName','CharityHeadquarters'])

    
    matches.to_csv('../generated/matches/officer/officer_' + leak +'_'+ charity +'_matches.csv')
    
    
    

In [5]:
# Extraction of possible charity names in the Panama Papers
extract_matches_between('panama', 'forbes', False)
extract_matches_between('panama', 'wikipedia', True)
extract_matches_between('panama', 'INGO', False)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Extraction of possible charity names in the Paradise Papers
extract_matches_between('paradise', 'forbes', False)
extract_matches_between('paradise', 'wikipedia', True)
extract_matches_between('paradise', 'INGO', False)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Extraction of possible charity names in the Bahamas Leaks
extract_matches_between('bahamas', 'forbes', False)
extract_matches_between('bahamas', 'wikipedia', True)
extract_matches_between('bahamas', 'INGO', False)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# Extraction of possible charity names in the Offshore Leaks
extract_matches_between('offshore', 'wikipedia', True)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 51.0 failed 1 times, most recent failure: Lost task 0.0 in stage 51.0 (TID 78, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/spark-2.3.2-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 253, in main
    process()
  File "/opt/spark/spark-2.3.2-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 248, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/spark-2.3.2-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 379, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/spark-2.3.2-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/util.py", line 55, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-4-1

In [14]:
extract_matches_between('offshore', 'INGO', False)
extract_matches_between('offshore', 'forbes', False)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
